# Combine workgraphs

## Introduction

There are two ways to combine workgraphs:

- Extending a workgraph by the tasks of another
- Adding a workgraph as a task of another

This tutorial will show both, highlighting the differences.


In [62]:
%load_ext aiida
from aiida import load_profile

load_profile()

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


Profile<uuid='36f11577e5ee4adf9e4a5f5d62d3ae9b' name='presto'>

In [63]:
from aiida_workgraph import WorkGraph, task

## Extending a workgraph by the tasks of another

We first define a workgraph to add two numbers and multiply the sum by a third.

In [64]:
@task
def add(a, b):
    return a + b


@task
def multiply(a, b):
    return a * b


@task.graph_builder
def add_multiply(a, b, c):
    wg = WorkGraph("AddMultiply")
    add_task = wg.add_task(add, a=a, b=b, name="add")
    wg.add_task(multiply, a=add_task.outputs.result, b=c, name="multiply")
    return wg

add_multiply_graph = add_multiply(1, 2, 3)

Next, we define a second workgraph with a task to generate a random number between 1 and 10. We then unpack the tasks of the first workgraph into the second, assigning the output of the random number generator as the factor of the multiplication task.

In [65]:
@task
def generate_random_number(minimum, maximum):
    import random

    return random.randint(minimum, maximum)


wg = WorkGraph("AddMultiplyExtension")
generate_number_task = wg.add_task(
    generate_random_number,
    minimum=1,
    maximum=10,
)
wg.extend(add_multiply_graph)
wg.add_link(
    generate_number_task.outputs.result,
    wg.tasks["multiply"].inputs.b,
)
wg.to_html()

Here we've authored a new workgraph as an extension of the first. The resultant workgraph reveals all tasks, with a clear dependency structure.

## Adding a workgraph as a task of another

Next, we will contrast this against adding the first workgraph as a task of the second.

In [66]:
wg = WorkGraph("AddMultiplyBlackBox")
random_number_generator = wg.add_task(
    generate_random_number,
    minimum=1,
    maximum=10,
)
add_multiply_task = wg.add_task(
    add_multiply,
    a=1,
    b=2,
    c=random_number_generator.outputs.result,
)
wg.to_html()

Here we've directly added the graph builder of the first workgraph as a task of the second. The resultant workgraph shows the random number generation task but does not show the internal dependency structure of the first workgraph. Instead, it is treated as a black box, with the output of the random number generator being passed to the graph builder input. The internal mechanism of the first workgraph is only made visible at runtime, when the first workgraph is executed.

## Summary

In this tutorial, we have seen how to combine workgraphs in two different ways: by extending a workgraph with the tasks of another, and by adding a workgraph as a black-box task. The choice between these methods depends on whether you want to see the internal structure of the combined workgraph or treat it as a black box, effectively trading off transparency and abstraction. Each method has its use cases, and understanding them will help you design more complex workflows in AiiDA.